### Trying to create a model for state by state data.
- First, combined the data from 2012, 2017, 2021 and 2021 to one dataframe.
- The states and year are combined to one States_year column. So, Alabama will have four rows: Alabama2012, Alabama2017, Alabama2021, Alabama2022
- The territories and DC have been excluded.
- The distribution by race has not been considered.
### Also did naïve modelling with statsmodels
- Used 10% of the 200 datasets as testing data and the rest as training data.
- Mean AP score is used as the target variable.
- Per capita income, population, density of R1/R2 universities (per million), density of public universities (per million) and density of private non-profit universities (per million) are used as features.
- Full model includes all features.
- Uni metric model includes density of R1/R2, public and private non-profit as features.
- Non-uni model includes per capita and population as features.
- Viewing non-uni model as null hypothesis, the p-value of full model is extremely low. So, null hypothesis should be excluded.
- Also, viewing uni metric model as null hypothesis, the p-value of full model is calculated to be extremely low. So, again, null hypothesis should be excluded.
- Prediction on testing data is computed. The root mean squared error is around 0.27. By comparison, majority of the mean AP scores are in 2.5-3 range.

In [313]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [314]:
carnegie = pd.read_excel('data/CCIHE2021-PublicData_limited.xlsx',sheet_name='Data')
ap_participation = pd.read_excel('data/AP_data_fixed-participation.xlsx',sheet_name='AP_data_for_reference')
ap_outcome = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='Sheet1')
us_income = pd.read_excel('data/per_capita_income_federal_reserve_stlouis.xlsx')
us_population = pd.read_excel('data/population_census_bureau.xlsx')

In [315]:
ap_passrate_22 = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='2022')
ap_passrate_17 = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='2017')
ap_passrate_21 = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='2021')
ap_passrate_12 = pd.read_excel('data/AP_data_fixed-outcome.xlsx',sheet_name='2012')

In [316]:
ap_passrate22=ap_passrate_22[ap_passrate_22['Race']=='Overall']  #Only pick 'Overall' scores for now, ignoring racial divisions.
ap_passrate21=ap_passrate_21[ap_passrate_21['Race']=='Overall']  #Only pick 'Overall' scores for now, ignoring racial divisions.
ap_passrate17=ap_passrate_17[ap_passrate_17['Race']=='Overall']  #Only pick 'Overall' scores for now, ignoring racial divisions.
ap_passrate12=ap_passrate_12[ap_passrate_12['Race']=='Overall']  #Only pick 'Overall' scores for now, ignoring racial divisions.

In [317]:
ap_passrate22['Year'] = 2022*np.ones(len(ap_passrate22.index))
ap_passrate21['Year'] = 2021*np.ones(len(ap_passrate21.index))
ap_passrate17['Year'] = 2017*np.ones(len(ap_passrate17.index))
ap_passrate12['Year'] = 2012*np.ones(len(ap_passrate12.index))


/tmp/ipykernel_845417/3380574313.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_845417/3380574313.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_845417/3380574313.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

In [318]:
ap_scores = pd.concat([ap_passrate22,ap_passrate21,ap_passrate17,ap_passrate12])
ap_scores

,State,Race,Score 5,Score 4,Score 3,Score 2,Score 1,Total,Mean,Year
0,National,Overall,576744,812854,990625,860182,810450,4050855,2.9,2022.0
9,Alabama,Overall,3539,6516,9838,11317,13031,44241,2.5,2022.0
18,Alaska,Overall,565,927,1323,1133,793,4741,2.9,2022.0
27,Arizona,Overall,9386,12578,15707,13518,11041,62230,2.9,2022.0
36,Arkansas,Overall,2553,5533,9253,10841,12925,41105,2.4,2022.0
...,...,...,...,...,...,...,...,...,...,...
423,Virginia,Overall,19593,27347,33203,29350,23645,133138,2.9,2012.0
432,Washington,Overall,9740,13107,16258,14170,11549,64824,2.9,2012.0
441,West Virginia,Overall,745,1321,2210,2924,2882,10082,2.4,2012.0
450,Wisconsin,Overall,7988,11973,14239,10073,6014,50287,3.1,2012.0


In [319]:
ap_scores.Year = ap_scores.Year.apply(np.int64)
ap_scores

,State,Race,Score 5,Score 4,Score 3,Score 2,Score 1,Total,Mean,Year
0,National,Overall,576744,812854,990625,860182,810450,4050855,2.9,2022
9,Alabama,Overall,3539,6516,9838,11317,13031,44241,2.5,2022
18,Alaska,Overall,565,927,1323,1133,793,4741,2.9,2022
27,Arizona,Overall,9386,12578,15707,13518,11041,62230,2.9,2022
36,Arkansas,Overall,2553,5533,9253,10841,12925,41105,2.4,2022
...,...,...,...,...,...,...,...,...,...,...
423,Virginia,Overall,19593,27347,33203,29350,23645,133138,2.9,2012
432,Washington,Overall,9740,13107,16258,14170,11549,64824,2.9,2012
441,West Virginia,Overall,745,1321,2210,2924,2882,10082,2.4,2012
450,Wisconsin,Overall,7988,11973,14239,10073,6014,50287,3.1,2012


In [320]:
exclude = ['National','District of Columbia']
ap_scores = ap_scores.drop(ap_scores[ap_scores['State'].isin(exclude)].index) 
ap_scores

,State,Race,Score 5,Score 4,Score 3,Score 2,Score 1,Total,Mean,Year
9,Alabama,Overall,3539,6516,9838,11317,13031,44241,2.5,2022
18,Alaska,Overall,565,927,1323,1133,793,4741,2.9,2022
27,Arizona,Overall,9386,12578,15707,13518,11041,62230,2.9,2022
36,Arkansas,Overall,2553,5533,9253,10841,12925,41105,2.4,2022
45,California,Overall,104767,127523,145798,124928,113544,616560,3,2022
...,...,...,...,...,...,...,...,...,...,...
423,Virginia,Overall,19593,27347,33203,29350,23645,133138,2.9,2012
432,Washington,Overall,9740,13107,16258,14170,11549,64824,2.9,2012
441,West Virginia,Overall,745,1321,2210,2924,2882,10082,2.4,2012
450,Wisconsin,Overall,7988,11973,14239,10073,6014,50287,3.1,2012


In [321]:
ap_scores['PERCENT_3_OR_ABOVE']=(ap_scores['Score 5']+ap_scores['Score 4']+ap_scores['Score 3'])*100/(ap_scores['Score 5']+ap_scores['Score 4']+ap_scores['Score 3']+ap_scores['Score 2']+ap_scores['Score 1'])
ap_scores.reset_index(drop=True,inplace=True)
ap_scores

,State,Race,Score 5,Score 4,Score 3,Score 2,Score 1,Total,Mean,Year,PERCENT_3_OR_ABOVE
0,Alabama,Overall,3539,6516,9838,11317,13031,44241,2.5,2022,44.965078
1,Alaska,Overall,565,927,1323,1133,793,4741,2.9,2022,59.375659
2,Arizona,Overall,9386,12578,15707,13518,11041,62230,2.9,2022,60.535112
3,Arkansas,Overall,2553,5533,9253,10841,12925,41105,2.4,2022,42.182216
4,California,Overall,104767,127523,145798,124928,113544,616560,3,2022,61.322175
...,...,...,...,...,...,...,...,...,...,...,...
195,Virginia,Overall,19593,27347,33203,29350,23645,133138,2.9,2012,60.195436
196,Washington,Overall,9740,13107,16258,14170,11549,64824,2.9,2012,60.32488
197,West Virginia,Overall,745,1321,2210,2924,2882,10082,2.4,2012,42.41222
198,Wisconsin,Overall,7988,11973,14239,10073,6014,50287,3.1,2012,68.009625


In [322]:
us_population.head()

,Regions,2012,2017,2021,2022
0,.Alabama,4815588,4874486,5050380,5073903
1,.Alaska,730443,739700,734923,733276
2,.Arizona,6554978,7044008,7272487,7365684
3,.Arkansas,2952164,3001345,3028443,3046404
4,.California,37948800,39358497,39145060,39040616


In [323]:
us_population['Regions']=us_population['Regions'].str.replace('.','')

In [324]:
us_population.head(n=10)

,Regions,2012,2017,2021,2022
0,Alabama,4815588,4874486,5050380,5073903
1,Alaska,730443,739700,734923,733276
2,Arizona,6554978,7044008,7272487,7365684
3,Arkansas,2952164,3001345,3028443,3046404
4,California,37948800,39358497,39145060,39040616
5,Colorado,5192647,5611885,5811596,5841039
6,Connecticut,3594547,3573297,3603691,3608706
7,Delaware,915179,956823,1004881,1019459
8,District of Columbia,634924,694906,669037,670949
9,Florida,19297822,20963613,21830708,22245521


In [325]:
us_population=us_population.drop([8])
us_population=us_population.reset_index()
us_population=us_population[['Regions',2022,2021,2017,2012]]

In [326]:
us_population.head(n=10)

,Regions,2022,2021,2017,2012
0,Alabama,5073903,5050380,4874486,4815588
1,Alaska,733276,734923,739700,730443
2,Arizona,7365684,7272487,7044008,6554978
3,Arkansas,3046404,3028443,3001345,2952164
4,California,39040616,39145060,39358497,37948800
5,Colorado,5841039,5811596,5611885,5192647
6,Connecticut,3608706,3603691,3573297,3594547
7,Delaware,1019459,1004881,956823,915179
8,Florida,22245521,21830708,20963613,19297822
9,Georgia,10913150,10790385,10410330,9901430


In [327]:
us_pop=pd.melt(us_population,id_vars='Regions',value_vars=[2022,2021,2017,2012],value_name='Population')
us_pop=us_pop.rename(columns={'Regions':'State','variable':'Year'})

In [328]:
us_pop

,State,Year,Population
0,Alabama,2022,5073903
1,Alaska,2022,733276
2,Arizona,2022,7365684
3,Arkansas,2022,3046404
4,California,2022,39040616
...,...,...,...
195,Virginia,2012,8185080
196,Washington,2012,6897058
197,West Virginia,2012,1856872
198,Wisconsin,2012,5719960


In [329]:
us_income['State']=us_income['State'].str.strip()

In [330]:
us_income.head(n=10)

,State,2022,2021,2017,2012
0,Alabama,51683,50483,39975,35564
1,Alaska,69015,65563,56499,53340
2,Arizona,58968,56976,43208,36333
3,Arkansas,55323,52845,41402,36287
4,California,76941,76882,58214,47794
5,Colorado,76674,71706,54171,45490
6,Connecticut,84994,81758,69146,63555
7,Delaware,63964,59630,50002,43775
8,District of Columbia,101015,97796,79535,67470
9,Florida,64557,62242,48439,41204


In [331]:
us_income=us_income.drop([8])
us_income=us_income.reset_index()
us_income=us_income[['State',2022,2021,2017,2012]]
us_income.head(n=10)

,State,2022,2021,2017,2012
0,Alabama,51683,50483,39975,35564
1,Alaska,69015,65563,56499,53340
2,Arizona,58968,56976,43208,36333
3,Arkansas,55323,52845,41402,36287
4,California,76941,76882,58214,47794
5,Colorado,76674,71706,54171,45490
6,Connecticut,84994,81758,69146,63555
7,Delaware,63964,59630,50002,43775
8,Florida,64557,62242,48439,41204
9,Georgia,57290,56088,44838,37251


In [332]:
us_pci = pd.melt(us_income,id_vars='State',value_vars=[2022,2021,2017,2012],value_name='Per_capita_income')
us_pci=us_pci.rename(columns={'variable':'Year'})
us_pci

,State,Year,Per_capita_income
0,Alabama,2022,51683
1,Alaska,2022,69015
2,Arizona,2022,58968
3,Arkansas,2022,55323
4,California,2022,76941
...,...,...,...
195,Virginia,2012,49052
196,Washington,2012,47057
197,West Virginia,2012,35233
198,Wisconsin,2012,42641


In [333]:
us_data=us_pci.merge(us_pop)
us_data

,State,Year,Per_capita_income,Population
0,Alabama,2022,51683,5073903
1,Alaska,2022,69015,733276
2,Arizona,2022,58968,7365684
3,Arkansas,2022,55323,3046404
4,California,2022,76941,39040616
...,...,...,...,...
195,Virginia,2012,49052,8185080
196,Washington,2012,47057,6897058
197,West Virginia,2012,35233,1856872
198,Wisconsin,2012,42641,5719960


In [334]:
ap_passrates=us_data.merge(ap_scores[['State','PERCENT_3_OR_ABOVE','Year']])
ap_passrates

,State,Year,Per_capita_income,Population,PERCENT_3_OR_ABOVE
0,Alabama,2022,51683,5073903,44.965078
1,Alaska,2022,69015,733276,59.375659
2,Arizona,2022,58968,7365684,60.535112
3,Arkansas,2022,55323,3046404,42.182216
4,California,2022,76941,39040616,61.322175
...,...,...,...,...,...
195,Virginia,2012,49052,8185080,60.195436
196,Washington,2012,47057,6897058,60.32488
197,West Virginia,2012,35233,1856872,42.41222
198,Wisconsin,2012,42641,5719960,68.009625


In [335]:
carnegie.head(n=10)

,unitid,name,city,stabbr,basic2000,basic2005,basic2010,basic2015,basic2018,basic2021,...,control,landgrnt,medical,hbcu,tribal,hsi,msi,womens,rooms,selindex
0,100654,Alabama A & M University,Normal,AL,16,18,18,18,18,18,...,1,1,0,1,0,0,1,0,3220,1.0
1,100663,University of Alabama at Birmingham,Birmingham,AL,15,15,15,15,15,15,...,1,0,1,0,0,0,0,0,2882,2.0
2,100690,Amridge University,Montgomery,AL,51,24,24,20,20,20,...,2,0,0,0,0,0,0,0,0,1.0
3,100706,University of Alabama in Huntsville,Huntsville,AL,16,16,15,16,16,15,...,1,0,0,0,0,0,0,0,2200,3.0
4,100724,Alabama State University,Montgomery,AL,21,18,18,19,19,17,...,1,0,0,1,0,0,1,0,2079,1.0
5,100751,The University of Alabama,Tuscaloosa,AL,15,16,16,16,15,15,...,1,0,0,0,0,0,0,0,8443,2.0
6,100760,Central Alabama Community College,Alexander City,AL,40,2,2,1,2,5,...,1,0,0,0,0,0,0,0,0,NaN
7,100812,Athens State University,Athens,AL,32,22,22,22,22,22,...,1,0,0,0,0,0,0,0,0,0.0
8,100830,Auburn University at Montgomery,Montgomery,AL,21,18,18,18,18,18,...,1,0,0,0,0,0,0,0,1200,1.0
9,100858,Auburn University,Auburn,AL,15,16,16,16,15,15,...,1,1,1,0,0,0,0,0,4823,3.0


In [336]:
print(carnegie.control.unique())
print(carnegie.basic2021.unique())

[1 2 3]
[18 15 20 17  5 22 21  4  2  7 26 24  8  1 19  3 16 32 23 13 10  9  6 33
 11 27 30 14 31 25 29 28 12]


In [337]:
territories = ['AS','GU','MP','FM','PW','VI','MH','PR','DC']  #US territories that appear in the data (including DC)

In [338]:
carnegie = carnegie.drop(carnegie[carnegie['stabbr'].isin(territories)].index)  #Drop all the states from the list "territories"

In [339]:
len(carnegie.stabbr.unique())

50

In [340]:
carnegie.columns

Index(['unitid', 'name', 'city', 'stabbr', 'basic2000', 'basic2005',
       'basic2010', 'basic2015', 'basic2018', 'basic2021', 'ipug2021',
       'ipgrad2021', 'obereg', 'sector', 'control', 'landgrnt', 'medical',
       'hbcu', 'tribal', 'hsi', 'msi', 'womens', 'rooms', 'selindex'],
      dtype='object')

In [341]:
relevant_columns = ['name','city','stabbr','basic2010','basic2015','basic2021','control']

In [342]:
carnegie = carnegie[relevant_columns]

In [343]:
state_abbreviation = pd.read_csv('data/State Abbreviation.csv')
state_abbreviation[state_abbreviation['stabbr'] == 'AL'].State

1    Alabama
Name: State, dtype: object

In [344]:
carnegie

,name,city,stabbr,basic2010,basic2015,basic2021,control
0,Alabama A & M University,Normal,AL,18,18,18,1
1,University of Alabama at Birmingham,Birmingham,AL,15,15,15,1
2,Amridge University,Montgomery,AL,24,20,20,2
3,University of Alabama in Huntsville,Huntsville,AL,15,16,15,1
4,Alabama State University,Montgomery,AL,18,19,17,1
...,...,...,...,...,...,...,...
3933,California University of Science and Medicine,Colton,CA,-2,-2,25,2
3934,California Institute of Arts & Technology-Nati...,National City,CA,-2,-2,11,3
3936,The Judge Advocate General's School,Charlottesville,VA,-2,-2,32,1
3937,United States Army War College,Carlisle,PA,-2,-2,32,1


In [345]:
def stabbr_to_state(state_abbrev):
    state = state_abbreviation[state_abbreviation['stabbr'] == state_abbrev].State.values[0]
    return state

In [346]:
carnegie['States']=carnegie['stabbr'].apply(stabbr_to_state)

In [347]:
carnegie

,name,city,stabbr,basic2010,basic2015,basic2021,control,States
0,Alabama A & M University,Normal,AL,18,18,18,1,Alabama
1,University of Alabama at Birmingham,Birmingham,AL,15,15,15,1,Alabama
2,Amridge University,Montgomery,AL,24,20,20,2,Alabama
3,University of Alabama in Huntsville,Huntsville,AL,15,16,15,1,Alabama
4,Alabama State University,Montgomery,AL,18,19,17,1,Alabama
...,...,...,...,...,...,...,...,...
3933,California University of Science and Medicine,Colton,CA,-2,-2,25,2,California
3934,California Institute of Arts & Technology-Nati...,National City,CA,-2,-2,11,3,California
3936,The Judge Advocate General's School,Charlottesville,VA,-2,-2,32,1,Virginia
3937,United States Army War College,Carlisle,PA,-2,-2,32,1,Pennsylvania


In [348]:
R1R2_2021 = {'States':[],'R1count':[],'R2count':[]}
for i in carnegie.States.unique():
    R1R2_2021['States'].append(i)
    R1R2_2021['R1count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2021.eq(15)]))
    R1R2_2021['R2count'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.basic2021.eq(16)]))

R1R2_2021=pd.DataFrame(R1R2_2021)
R1R2_2021['R1plusR2']=R1R2_2021.R1count+R1R2_2021.R2count
R1R2_2021.head(n=10)

,States,R1count,R2count,R1plusR2
0,Alabama,4,1,5
1,Alaska,0,1,1
2,Washington,2,0,2
3,Arizona,2,2,4
4,New Mexico,1,1,2
5,Arkansas,1,2,3
6,California,11,14,25
7,Minnesota,1,0,1
8,Colorado,5,1,6
9,Connecticut,2,0,2


In [349]:
public_private = {'States':[],'public':[],'private_not_profit':[],'private_for_profit':[]}
for i in carnegie.States.unique():
    public_private['States'].append(i)
    public_private['public'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(1)]))
    public_private['private_not_profit'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(2)]))
    public_private['private_for_profit'].append(len(carnegie.loc[carnegie.States.eq(i) & carnegie.control.eq(3)]))

public_private = pd.DataFrame(public_private)
public_private

,States,public,private_not_profit,private_for_profit
0,Alabama,38,18,4
1,Alaska,4,3,1
2,Washington,42,23,7
3,Arizona,26,13,27
4,New Mexico,28,3,5
5,Arkansas,34,17,2
6,California,153,151,121
7,Minnesota,45,33,7
8,Colorado,29,9,24
9,Connecticut,19,17,2


In [350]:
unicount22=public_private.merge(R1R2_2021)
unicount22['Year'] = 2022*np.ones(len(unicount22.index))
unicount21=public_private.merge(R1R2_2021)
unicount21['Year'] = 2021*np.ones(len(unicount21.index))
unicount17=public_private.merge(R1R2_2021)
unicount17['Year'] = 2017*np.ones(len(unicount17.index))
unicount12=public_private.merge(R1R2_2021)
unicount12['Year'] = 2012*np.ones(len(unicount12.index))
unicount=pd.concat([unicount22,unicount21,unicount17,unicount12])
unicount.reset_index(drop=True,inplace=True)
unicount

,States,public,private_not_profit,private_for_profit,R1count,R2count,R1plusR2,Year
0,Alabama,38,18,4,4,1,5,2022.0
1,Alaska,4,3,1,0,1,1,2022.0
2,Washington,42,23,7,2,0,2,2022.0
3,Arizona,26,13,27,2,2,4,2022.0
4,New Mexico,28,3,5,1,1,2,2022.0
...,...,...,...,...,...,...,...,...
195,Vermont,5,11,0,0,1,1,2012.0
196,Virginia,42,40,26,5,2,7,2012.0
197,West Virginia,22,9,10,1,1,2,2012.0
198,Wisconsin,33,30,4,2,1,3,2012.0


In [351]:
unicount.Year = unicount.Year.apply(np.int64)
unicount = unicount[['States','public','private_not_profit','R1plusR2','Year']]

In [352]:
unicount=unicount.rename(columns={'States':'State'})

In [353]:
ap_passrates

,State,Year,Per_capita_income,Population,PERCENT_3_OR_ABOVE
0,Alabama,2022,51683,5073903,44.965078
1,Alaska,2022,69015,733276,59.375659
2,Arizona,2022,58968,7365684,60.535112
3,Arkansas,2022,55323,3046404,42.182216
4,California,2022,76941,39040616,61.322175
...,...,...,...,...,...
195,Virginia,2012,49052,8185080,60.195436
196,Washington,2012,47057,6897058,60.32488
197,West Virginia,2012,35233,1856872,42.41222
198,Wisconsin,2012,42641,5719960,68.009625


In [354]:
ap_overall_statelevel=unicount.merge(ap_passrates)

In [355]:
ap_overall_statelevel['R1R2_per_million'] = ap_overall_statelevel['R1plusR2']*1000000/ap_overall_statelevel['Population']
ap_overall_statelevel['public_per_million'] = ap_overall_statelevel['public']*1000000/ap_overall_statelevel['Population']
ap_overall_statelevel['private_notprofit_per_million'] = ap_overall_statelevel['private_not_profit']*1000000/ap_overall_statelevel['Population']

ap_overall_statelevel

,State,public,private_not_profit,R1plusR2,Year,Per_capita_income,Population,PERCENT_3_OR_ABOVE,R1R2_per_million,public_per_million,private_notprofit_per_million
0,Alabama,38,18,5,2022,51683,5073903,44.965078,0.985435,7.489304,3.547565
1,Alaska,4,3,1,2022,69015,733276,59.375659,1.363743,5.454972,4.091229
2,Washington,42,23,2,2022,75673,7784477,65.911295,0.256922,5.395353,2.954598
3,Arizona,26,13,4,2022,58968,7365684,60.535112,0.543059,3.529883,1.764941
4,New Mexico,28,3,2,2022,53079,2113476,40.522055,0.946308,13.248317,1.419463
...,...,...,...,...,...,...,...,...,...,...,...
195,Vermont,5,11,1,2012,44861,626090,65.373587,1.597214,7.986072,17.569359
196,Virginia,42,40,7,2012,49052,8185080,60.195436,0.855215,5.131288,4.886941
197,West Virginia,22,9,2,2012,35233,1856872,42.41222,1.077080,11.847882,4.846861
198,Wisconsin,33,30,3,2012,42641,5719960,68.009625,0.524479,5.769271,5.244792


In [356]:
features = ['Per_capita_income','R1R2_per_million','public_per_million','private_notprofit_per_million','Population']
ap_overall_statelevel[features]

,Per_capita_income,R1R2_per_million,public_per_million,private_notprofit_per_million,Population
0,51683,0.985435,7.489304,3.547565,5073903
1,69015,1.363743,5.454972,4.091229,733276
2,75673,0.256922,5.395353,2.954598,7784477
3,58968,0.543059,3.529883,1.764941,7365684
4,53079,0.946308,13.248317,1.419463,2113476
...,...,...,...,...,...
195,44861,1.597214,7.986072,17.569359,626090
196,49052,0.855215,5.131288,4.886941,8185080
197,35233,1.077080,11.847882,4.846861,1856872
198,42641,0.524479,5.769271,5.244792,5719960


In [360]:
ap_overall_statelevel = ap_overall_statelevel.apply(pd.to_numeric, errors='coerce')

In [384]:
import plotly.express as px
import plotly.graph_objects as go
import pickle

In [385]:
fig = px.scatter(data_frame=ap_overall_statelevel,x='Per_capita_income', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs Per Capita Income')
fig.show()
with open('data/state_by_state_pickled/AP_pass_rate_by_states_vs_income.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [387]:
fig = px.scatter(data_frame=ap_overall_statelevel,x='R1R2_per_million', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs R1R2 uni count per million')
fig.show()
with open('data/state_by_state_pickled/AP_pass_rate_by_states_vs_r1r2count.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [388]:
fig = px.scatter(data_frame=ap_overall_statelevel,x='public_per_million', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs Public uni count per million')
fig.show()
with open('data/state_by_state_pickled/AP_pass_rate_by_states_vs_Public_uni_count.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [389]:
fig = px.scatter(data_frame=ap_overall_statelevel,x='Population', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs Population')
fig.show()
with open('data/state_by_state_pickled/AP_pass_rate_by_states_vs_population.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [390]:
fig = px.scatter(data_frame=ap_overall_statelevel,x='private_notprofit_per_million', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs Private uni count per million')
fig.show()
with open('data/state_by_state_pickled/AP_pass_rate_by_states_vs_private_uni_count.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [361]:
from sklearn.model_selection import train_test_split
training, testing = train_test_split(ap_overall_statelevel, test_size = 0.1, random_state = 216)

In [362]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [363]:
full_model = smf.ols('PERCENT_3_OR_ABOVE ~ Per_capita_income + R1R2_per_million + public_per_million + private_notprofit_per_million + Population', data=training).fit()
pci_model = smf.ols('PERCENT_3_OR_ABOVE ~ Per_capita_income', data =training).fit()
r1r2_model = smf.ols('PERCENT_3_OR_ABOVE ~ R1R2_per_million', data = training).fit()
unimetric_model = smf.ols('PERCENT_3_OR_ABOVE ~ R1R2_per_million + public_per_million + private_notprofit_per_million', data = training).fit()
nonuni_model = smf.ols('PERCENT_3_OR_ABOVE ~ Per_capita_income + Population', data=training).fit()

In [364]:
full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     PERCENT_3_OR_ABOVE   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     10.83
Date:                Wed, 27 Nov 2024   Prob (F-statistic):           4.42e-09
Time:                        11:05:08   Log-Likelihood:                -630.14
No. Observations:                 180   AIC:                             1272.
Df Residuals:                     174   BIC:                             1291.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        38.6720      3.368     11.480      0.000      32.024      45.320
Per_capita_income                 0.0003   5.37e-05      4.663      0.000       0.000       0.000
R1R2_per_million                  2.3227      1.554      1.495      0.137      -0.745       5.390
public_per_million               -0.3588      0.195     -1.836      0.068      -0.745       0.027
private_notprofit_per_million     0.8365      0.208      4.027      0.000       0.427       1.246
Population                     3.071e-08   1.01e-07      0.305      0.761   -1.68e-07    2.29e-07
==============================================================================
Omnibus:                        5.563   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                4.900
Skew:                          -0.326   Prob(JB):                       0.0863
Kurtosis:                       2.523   Cond. No.                     5.16e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.16e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [365]:
f_test_nonuni = full_model.compare_f_test(nonuni_model)
f_test_unimetric = full_model.compare_f_test(unimetric_model)
print("p-value of full compared to non-university metric model:", f_test_nonuni[1])
print("p-value of full compared to university metric model:", f_test_unimetric[1])


p-value of full compared to non-university metric model: 3.2505521080123143e-05
p-value of full compared to university metric model: 2.167572819031781e-05


### The p-values are extremely small. So, it seems both university and non-university metrics are essential for good modeling. So, we choose full model as our final model and use it on the testing data.

In [366]:
predictions = full_model.predict(testing)  # Compute the predictions on the testing data

In [381]:
from statsmodels.tools.eval_measures import rmse

rms_error = rmse(testing['PERCENT_3_OR_ABOVE'], predictions)
print('root mean squared error is: ',rms_error)

root mean squared error is:  9.615816084229623


In [378]:
avgs= np.mean(testing['PERCENT_3_OR_ABOVE'])*np.ones(len(testing.index))

In [379]:
np.mean(predictions)

np.float64(58.04488102892551)

In [380]:
rmse(avgs,testing['PERCENT_3_OR_ABOVE'])

np.float64(10.881659865054784)